In [15]:
import torch
import time
from torchvision import datasets , transforms

In [2]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor()])

In [3]:
train_dataset = datasets.ImageFolder("./Dataset/train/",transform=train_transform)
test_dataset  = datasets.ImageFolder("./Dataset/valid/",transform=test_transform)

In [4]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size=128,shuffle=True)

In [5]:
from torchvision import models
model = models.resnet18(pretrained = True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [6]:
from collections import OrderedDict

for params in model.parameters():
    params.requires_grad = False

classifier = torch.nn.Sequential(OrderedDict([
            ('fc1',torch.nn.Linear(512,128)),
            ('relu',torch.nn.ReLU()),
            ('fc2',torch.nn.Linear(128,64)),
            ('relu2',torch.nn.ReLU()),
            ('fc3',torch.nn.Linear(64,2)),
            ('output',torch.nn.LogSoftmax(dim=1))
]))
model.fc = classifier
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.fc.parameters(),lr=0.003)

In [9]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
epoch = 5
steps = 0
train_loss =[]
test_loss = []
for e in range(epoch):
    running_loss=0
    val_loss = 0
    acc= 0
    t1 = time.time()
    t2 =0
    for images , labels in trainloader:
        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        log_ps = model.forward(images)
        loss = criterion(log_ps,labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        t2 = time.time()
            
        
    else:
        with torch.no_grad():
            model.eval()
            
            for images,labels in testloader:
                images,labels = images.to(device),labels.to(device)
                log_ps = model.forward(images)
                ps = torch.exp(log_ps)
                loss = criterion(log_ps,labels)
                val_loss += loss.item()
                _,top_class = ps.topk(1,dim=1)
                equal = top_class == labels.view(*top_class.shape)
                
                acc += torch.mean(equal.type(torch.FloatTensor)).item()
                
    model.train()
    train_loss.append(running_loss/len(trainloader))
    test_loss.append(val_loss/len(testloader))
    
    print("Epoch : {}/{}".format(e+1,epoch),
         "Train Loss : {:.3f}".format(running_loss/len(trainloader)),
         "Val Loss : {:.3f}".format(val_loss/len(testloader)),
         "Val acc : {:.3f}".format((acc/len(testloader))*100),
         "Time Elapsed : {:.3f}".format(t2-t1))
            

Epoch : 1/5 Train Loss : 0.168 Val Loss : 0.054 Val acc : 97.988 Time Elapsed : 346.405


In [11]:
torch.save(model.state_dict(), './model.pth')